In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

In [ ]:
!pip install konlpy

In [ ]:
# test dataset upload
from google.colab import files
myfile = files.upload()

In [ ]:
!unzip kordata.zip

In [ ]:
data = pd.read_csv('kordata.csv', encoding = 'cp949')

In [ ]:
data.head(3)

In [ ]:
data2 = list(data.document)
data2[0:2]

In [ ]:
##구두점 제거
from string import punctuation
def strip_pumctuation(s):
    return ''.join(c for c in s if c not in punctuation)

clean_title = []
for sent in data2 :
    clean = strip_pumctuation(sent)
    clean_title.append(clean)

clean_title[0:1]                 


In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()

In [ ]:
import time
import datetime
start = time.time()

In [ ]:
#데이터 프레임의 'Sentence' 열의 값들을 str 형식으로 바꾸기
data.document = data.document.astype(str)

#데이터 프레임의 'Sentence' 열의 값 중 keyword1이나 keyword 2가 포함된 행만 Data에 저장
clean_data = data.loc[data['document'].str.contains('keyword1|keyword2')]

#데이터 프레임의 'Sentence' 열의 값 중 keyword1이나 keyword 2가 포함된 행은 삭제
clean_data = data[data['document'].str.contains('keyword1|keyword2')]


#Sentence와 열을 기준으로 중복된 데이터를 삭제, inplace : 데이터 프레임을 변경할지 선택(원본을 대체)
clean_data.drop_duplicates(subset=['document'], inplace=True)

#한글이 아니면 빈 문자열로 바꾸기
clean_data['document'] = clean_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',regex=True)

#빈 문자열 NAN 값으로 바꾸기
clean_data = clean_data.replace({'': np.nan})
clean_data = clean_data.replace(r'^\s*$', None, regex=True)

#NAN 이 있는 행은 삭제
clean_data.dropna(how='any', inplace=True)

#인덱스 차곡차곡
clean_data = clean_data.reset_index (drop = True)

#데이터 프레임에 null 값이 있는지 확인
print(clean_data.isnull().values.any()) 

In [ ]:
dataset = []
for i in range(len(clean_title)) :
      dataset.append(kkma.nouns(re.sub('[^가-힣a-zA-Z\s]', '' , clean_title[i])))


sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)),split(".")
times = times[0]
print(times)

In [ ]:
clean_title2 = []
for i in range(len(clean_title)) :
    clean_title2.append(re.sub('영화','',clean_title[i]))

clean_title3 = []
for i in range(len(clean_title2)) :
    clean_title2.append(re.sub('내용','',clean_title2[i]))

In [ ]:
!pip install gensim
from gensim import corpora, models
import gensim

In [ ]:
high_score_reviews = dataset

In [ ]:
high_score_reviews = [[y for y in x if not len(y)==1]
                      for x in high_score_reviews]
dictionary = corpora.Dictionary(high_score_reviews)
corpus = [dictionary.doc2bow(text) for text in high_score_reviews]

In [ ]:
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel

coherence_values = []
for i in range(2,15) :
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = i, id2word = dictionary)
    coherence_model_lda = CoherenceModel(model = ldamodel, texts = high_score_reviews, dictionary=dictionary, topn=10)
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)

In [ ]:
x = range(2,15)
plt.plot(x,coherence_values)
plt.xlabel("number of topics")
plt.ylabel("coherence score")
plt.show()          

In [ ]:
import matplotlib.pyplot as plt
perplexity_values = []
for i in range(2,20) :
    Idamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=dictionary)
    perplexity_values.append(ldamodel.log_perplexity(corpus))


In [ ]:
x=range(2,20)
plt.plot(x,perplexity_values)
plt.xlabel("number of topics")
plt.ylabel("perplexity score")
plt.show()      

In [ ]:
# 결과 바탕으로 갯수 수정
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=6, alpha=0.1,
                                           id2word=dictionary)

In [ ]:
Idamodel.print_topics(num_words=20)

In [ ]:
pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis

In [ ]:
kk=ldamodel.show_topic(3,topn=90000)
type(ldamodel.show_topic)
kk2=pd.DataFrame(kk)
kk2.head(10)


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive') 

In [ ]:
kk2.to_csv('test8.csv',encoding="euc-kr")

In [ ]:
from google.colab import files

files.download('test8.csv')

In [ ]:
def make_topictable_per_doc(ldamodel, corpus) :
    topic_table = pd.DataFrame()

    for i, topic_list in enumerate(ldamodel[corpus]) :
        doc=topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]),reverse=True)
        #각문서에 대해 비중이 높은 토픽순으로 정렬

    #모든문서에 대해 수행
        for j, (topic_num, prop_topic) in enumerate(doc) :
           if j == 0:
               topic_table = topic_table.append(pd.Series([int(topic_num),round(prop_topic,4), topic_list]), ignore_index=True)
           else:
              break
    return(topic_table)
       

In [ ]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index()
topictable.columns = ['문서번호', '가장비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']


In [ ]:
topictable.to_csv("topdoc1.csv", encoding="euc-kr",index=False)

In [ ]:
from google.colab import files

files.download('topdoc1.csv')